In [13]:
import dowhy
from dowhy import CausalModel
from IPython.display import Image, display
import pandas as pd

# a utility function to parse the .gml file to string
def gml_to_string(file):
    gml_str = ''
    with open(file, 'r') as file:
        for line in file:
            gml_str += line.rstrip()
    return gml_str


def dowhy_backdoor(file, treat, out):

    # creating arbitrary data
    df = pd.read_stata("data/close_college.dta")
    df=df.dropna()

    gml_graph = gml_to_string(file)
    # With GML string
    model=CausalModel(
        data = df,
        treatment=treat,
        outcome=out,
        graph=gml_graph
    )

    identified_estimand = model.identify_effect()
    # print(identified_estimand)
    return model
# model = dowhy_backdoor('graph_files/dag1.gml')


In [19]:
import os
def prove_causality(file,  treat, out):
    model = dowhy_backdoor(file,  treat, out)
    identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
    adjustment_set = identified_estimand.get_backdoor_variables()
    causal_estimate = model.estimate_effect(identified_estimand,
        method_name="backdoor.linear_regression")
    
    return adjustment_set, causal_estimate.value


def get_causal_df_for_all_DAGs(folder, treat, out):
    files = os.listdir(folder)

    causal_df = pd.DataFrame(columns=['DAG', 'estimate', 'adjustment_set'])
    
    for file in files:
        adj_set, estimate = prove_causality(f'{folder}/{file}',  treat, out)
        causal_df.loc[len(causal_df)] = {'DAG':file, 'estimate': estimate, 'adjustment_set':adj_set}

    return causal_df


#### PC with prior knowledge CPDAG

In [23]:
df_estimate = get_causal_df_for_all_DAGs("graph_files/PC_CPDAG_graphs", 'educ', 'lwage')
drop_list = [3,5,9]
df_estimate = df_estimate.drop(drop_list)

display(df_estimate)

,DAG,estimate,adjustment_set
0,dag1.gml,0.079080,"[nearc4, exper, black]"
1,dag10.gml,0.079080,"[nearc4, exper, black]"
2,dag11.gml,0.074716,"[nearc4, exper, black]"
4,dag2.gml,0.079080,"[nearc4, exper, black]"
6,dag4.gml,0.079080,"[nearc4, exper, black]"
7,dag5.gml,0.079080,"[nearc4, exper, black]"
8,dag6.gml,0.077290,"[nearc4, exper, black]"
10,dag8.gml,0.076120,"[nearc4, exper, black]"
11,dag9.gml,0.076120,"[nearc4, exper, black]"


In [24]:
df_estimate['estimate'].mean()

### PC without prior knowledge CPDAG

In [16]:
get_causal_df_for_all_DAGs("graph_files/PC_withoutPK_CPDAG_graphs", 'educ', 'lwage')

,DAG,estimate,adjustment_set
0,dag1.gml,0,[]
1,dag10.gml,0,[]
2,dag11.gml,0,[]
3,dag12.gml,0,[]
4,dag2.gml,0,[]
5,dag3.gml,0,[]
6,dag4.gml,0,[]
7,dag5.gml,0,[]
8,dag6.gml,0,[]
9,dag7.gml,0,[]


In [17]:
get_causal_df_for_all_DAGs("graph_files/PC_withoutPK_CPDAG_graphs", 'lwage', 'educ')

,DAG,estimate,adjustment_set
0,dag1.gml,1.741521,"[nearc4, black]"
1,dag10.gml,1.742055,"[nearc4, black]"
2,dag11.gml,1.742055,"[nearc4, black]"
3,dag12.gml,1.742055,"[nearc4, black]"
4,dag2.gml,1.741521,"[nearc4, black]"
5,dag3.gml,1.741521,"[nearc4, black]"
6,dag4.gml,1.741521,"[nearc4, black]"
7,dag5.gml,1.742055,"[nearc4, black]"
8,dag6.gml,1.742055,"[nearc4, black]"
9,dag7.gml,1.742055,"[nearc4, black]"


#### GES

In [18]:
get_causal_df_for_all_DAGs("graph_files/GES_CPDAG_graphs", 'lwage', 'educ')

,DAG,estimate,adjustment_set
0,dag1.gml,1.970842,[]
1,dag10.gml,1.970842,[]
2,dag11.gml,1.726602,[smsa]
3,dag12.gml,1.913223,"[smsa, married]"
4,dag13.gml,1.897900,[]
5,dag14.gml,1.897900,[]
6,dag15.gml,1.931013,[smsa]
7,dag16.gml,1.913223,"[smsa, married]"
8,dag17.gml,1.726602,[smsa]
9,dag18.gml,1.921437,"[smsa, married]"
